<a href="https://colab.research.google.com/github/wonrax/phobert-base-vietnamese-sentiment/blob/add-notebook/phobert_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch

In [ ]:
%pip install transformers
%pip install vncorenlp

!git clone https://github.com/vncorenlp/VnCoreNLP.git

     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 32.8 MB/s 
     |████████████████████████████████| 6.6 MB 35.0 MB/s 
     |████████████████████████████████| 880 kB 36.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=a5520023df2726cb700c2afec417c75942782e7b4ccc1bd78c4d935061daa837
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


     |████████████████████████████████| 2.6 MB 5.4 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=88ef8fd4148dfdbf9f289303a25bc72bb20df261b40d70bf79fb1555a1a8ed54
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp
Cloning into 'VnCoreNLP'...
remote: Enumerating objects: 215, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 215 (delta 0), reused 2 (delta 0), pack-reused 212
Receiving objects: 100% (215/215), 214.22 MiB | 32.51 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [ ]:
train_data_csv = "reviews-ecommerce.csv"
dataset = pd.read_csv(train_data_csv)
dataset

,comment,label,rate,Unnamed: 3
0,Áo bao đẹp ạ!!,POS,5,NaN
1,Tuyệt vời !,POS,5,NaN
2,2day ao khong giong trong.,NEG,1,NaN
3,"Mùi thơm,bôi lên da mềm da.",POS,5,NaN
4,"Vải đẹp, dày dặn.",POS,5,NaN
...,...,...,...,...
31455,Không đáng tiền.,NEG,1,NaN
31456,Quần rất đẹp.,POS,5,NaN
31457,Hàng đẹp đúng giá tiền.,POS,5,NaN
31458,Chất vải khá ổn.,POS,4,NaN


In [ ]:
# Perform preprocessing
import re

# TODO use a teen code list
rep = {"k": "không", "ko": "không", "kh": "không", "đc": "được", "dc": "được", "a": "anh", "e": "em", "v": "vậy", "vs": "với", "m": "mình"} # define desired replacements here

# use these three lines to do the replacement
rep_ = dict((r"\b{}\b".format(k), v) for k, v in rep.items())
pattern = re.compile("|".join(rep_.keys()), flags=re.I)

dataset["comment"] = dataset["comment"].apply(lambda text: pattern.sub(lambda m: rep[re.escape(m.group(0)).lower()], text))

dataset

NameError: ignored

In [ ]:
# Split dataset into training and test sets
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(dataset["comment"].values, dataset["label"].values, test_size=0.15, random_state=42)
train_texts, valid_texts, train_labels, valid_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=232)

print(train_texts.shape, valid_texts.shape, test_labels.shape)
print(train_texts[:5], train_labels[:5])
print(valid_texts[:5], valid_labels[:5])
print(test_texts[:5], test_labels[:5])

(21392,) (5349,) (4719,)
['Shop giao hàng nhanh như chớp luôn.' 'Đóng gói sp cẩn thận.'
 'Đợi test thử vài ngày nữa xem sao !'
 'Shop phục vụ rất tốt .Rất đáng tiền.'
 'Áo hơi mỏng nhưng mặc đẹp, dây của áo hoàn thiện không được tốt cho lắm, mong shop khắc phục.'] ['POS' 'POS' 'NEU' 'NEU' 'POS']
['Chất lượng sản phẩm tuyệt vời.' 'Đặt màu vàng thì ra màu gì vậy shop???'
 'Tôi không hề nhận được sản phẩm này.' 'Áo mỏng hơn ảnh chụp.' 'Tuyệt.'] ['POS' 'NEG' 'NEG' 'NEG' 'POS']
['Máy chụp rất ok.'
 'Chất lượng sản phẩm tuyệt vời Đóng gói sản phẩm rất đẹp và chắc chắn Đóng gói sản phẩm rất đẹp và chắc chắn Rất đáng tiền Thời gian giao hàng rất nhanh , yêu shop.'
 'Chất lượng tam duoc.' 'Hang rât tôt ủng hộ dài dài.' 'Vải cũng đẹp nữa.'] ['POS' 'POS' 'NEG' 'POS' 'POS']


In [ ]:
from transformers import AutoTokenizer
from vncorenlp import VnCoreNLP

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", cache_dir="../phobert-base")
rdrsegmenter = VnCoreNLP(
    "./VnCoreNLP/VnCoreNLP-1.1.1.jar",
    annotators="wseg,pos",
    max_heap_size="-Xmx2g",
)

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# PREPARE DATA BEFORE FEEDING TO MODEL

# Segmentize
train_segmented = [rdrsegmenter.tokenize(text) for text in train_texts]
valid_segmented = [rdrsegmenter.tokenize(text) for text in valid_texts]
test_segmented = [rdrsegmenter.tokenize(text) for text in test_texts]

# Flatten and merge back into strings
def flatten(t):
    return [item for sublist in t for item in sublist]
train_segmented = [" ".join(flatten(t)) for t in train_segmented]
valid_segmented = [" ".join(flatten(t)) for t in valid_segmented]
test_segmented = [" ".join(flatten(t)) for t in test_segmented]

print(train_segmented[:5], valid_segmented[:5], test_segmented[:5])

['Shop giao hàng nhanh như chớp luôn .', 'Đóng_gói sp cẩn_thận .', 'Đợi test thử vài ngày nữa xem sao !', 'Shop phục_vụ rất tốt . Rất đáng tiền .', 'Áo hơi mỏng nhưng mặc đẹp , dây của áo hoàn_thiện không được tốt cho lắm , mong shop khắc_phục .'] ['Chất_lượng sản_phẩm tuyệt_vời .', 'Đặt màu vàng thì ra màu gì vậy shop ? ? ?', 'Tôi không hề nhận được sản_phẩm này .', 'Áo mỏng hơn ảnh chụp .', 'Tuyệt .'] ['Máy chụp rất ok .', 'Chất_lượng sản_phẩm tuyệt_vời Đóng_gói sản_phẩm rất đẹp và chắc_chắn Đóng_gói sản_phẩm rất đẹp và chắc_chắn Rất đáng tiền Thời_gian giao hàng rất nhanh , yêu shop .', 'Chất_lượng tam duoc .', 'Hang rât tôt ủng_hộ dài_dài .', 'Vải cũng đẹp nữa .']


In [ ]:
# Tokenize
train_encodings = tokenizer(train_segmented, truncation=True, padding="max_length", max_length=256)
valid_encodings = tokenizer(valid_segmented, truncation=True, padding="max_length", max_length=256)
test_encodings = tokenizer(test_segmented, truncation=True, padding="max_length", max_length=256)

In [ ]:
print(train_segmented[2])
print(train_encodings["input_ids"][2][:20])
print(train_encodings["attention_mask"][2][:20])

Đợi test thử vài ngày nữa xem sao !
[0, 15621, 13923, 1176, 515, 43, 348, 305, 423, 381, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
import torch

class EcommReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
        self.label_to_idx = {'NEG': 0, 'POS': 1, 'NEU': 2}
        self.idx_to_label = {idx: label for label, idx in self.label_to_idx.items()}

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.label_to_idx[self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = EcommReviewsDataset(train_encodings, train_labels)
val_dataset = EcommReviewsDataset(valid_encodings, valid_labels)
test_dataset = EcommReviewsDataset(test_encodings, test_labels)

In [ ]:
# INIT MODEL

from transformers import AutoModelForSequenceClassification


## Set load location here
where_to_load = "local"

num_labels = 3
# Specify locally saved model (will be ignored if loading pretrained)
model_dir = '/content/drive/MyDrive/thesis/models/ecomm-review-phobert-base-5epochs'

model = None

if where_to_load == "local":
  model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=num_labels)
elif where_to_load == "pretrained":
  model_dir = "vinai/phobert-base"
  model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=num_labels, cache_dir="./phobert-base")
else:
  checkpoint_dir = "/content/drive/MyDrive/thesis/models/ecomm-review-phobert-base-4epochs/checkpoints/checkpoint-2500"
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint_dir, num_labels=num_labels)
assert model is not None

In [ ]:
!pip install huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
model.push_to_hub("phobert-base-vietnamese-sentiment")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:43: FutureWarning: Pass token='phobert-base-vietnamese-sentiment' as keyword args. From version 0.7 passing these as positional arguments will result in an error
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:599: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/wonrax/phobert-base-vietnamese-sentiment into local empty directory.


Upload file pytorch_model.bin:   0%|          | 3.34k/515M [00:00<?, ?B/s]

To https://huggingface.co/wonrax/phobert-base-vietnamese-sentiment
   9de30a1..eb3e629  main -> main



'https://huggingface.co/wonrax/phobert-base-vietnamese-sentiment/commit/eb3e629cc4a9ff3a83c0650c4d1aa790611ab6cf'

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", cache_dir="../phobert-base")
tokenizer.push_to_hub("phobert-base-vietnamese-sentiment")

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
To https://huggingface.co/wonrax/phobert-base-vietnamese-sentiment
   eb3e629..d2a1b77  main -> main



'https://huggingface.co/wonrax/phobert-base-vietnamese-sentiment/commit/d2a1b7722ef968abb933c4cd04ef059b8b43793d'

In [ ]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-325afcb5-37aa-132d-a870-454b98d24eee)


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
for name, p in model.named_parameters():
  print(p.shape, name)

NameError: ignored

In [ ]:
from transformers import Trainer, TrainingArguments

torch.cuda.empty_cache()

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/thesis/models/ecomm-review-phobert-base-5epochs/checkpoints',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=2,  # batch size per device during training
    per_device_eval_batch_size=2,   # batch size for evaluation
    warmup_steps=200,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
    learning_rate=6e-8,
#    fp16=True,                       # use 16bit float, train faster
)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=val_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

if where_to_load == "checkpoint":
  print("Training from checkpoint")
  trainer.train(checkpoint_dir)
elif where_to_load == "local" or where_to_load == "pretrained":
  trainer.train()

Loading model from /content/drive/MyDrive/thesis/models/ecomm-review-phobert-base-4epochs/checkpoints/checkpoint-2500).
You are resuming training from a checkpoint trained with 4.17.0 of Transformers but your current version is 4.18.0. This is not recommended and could yield to errors or unwanted behaviors.


Training from checkpoint


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5349
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 8025
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 0
  Continuing training from global step 2500
  Will skip the first 0 epochs then the first 2500 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/2500 [00:00<?, ?it/s]

Step,Training Loss
2600,0.558500
2700,0.589100
2800,0.611300
2900,0.716500
3000,0.689700
3100,0.631800
3200,0.715800
3300,0.476400
3400,0.716600
3500,0.599000


Saving model checkpoint to /content/drive/MyDrive/thesis/models/ecomm-review-phobert-base-5epochs/checkpoints/checkpoint-3000
Configuration saved in /content/drive/MyDrive/thesis/models/ecomm-review-phobert-base-5epochs/checkpoints/checkpoint-3000/config.json
Model weights saved in /content/drive/MyDrive/thesis/models/ecomm-review-phobert-base-5epochs/checkpoints/checkpoint-3000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/thesis/models/ecomm-review-phobert-base-5epochs/checkpoints/checkpoint-3500
Configuration saved in /content/drive/MyDrive/thesis/models/ecomm-review-phobert-base-5epochs/checkpoints/checkpoint-3500/config.json
Model weights saved in /content/drive/MyDrive/thesis/models/ecomm-review-phobert-base-5epochs/checkpoints/checkpoint-3500/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/thesis/models/ecomm-review-phobert-base-5epochs/checkpoints/checkpoint-4000
Configuration saved in /content/drive/MyDrive/thesis/models/ecomm-review-p

KeyboardInterrupt: ignored

In [ ]:
# Evaluation on test set

model.eval()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# test_dataset[0]['input_ids'].shape

for i in range(502, 700, 10):
  result = model(input_ids=test_dataset[i]["input_ids"].unsqueeze(0).to(device),
        attention_mask=test_dataset[i]["attention_mask"].unsqueeze(0).to(device),
        token_type_ids=test_dataset[i]["token_type_ids"].unsqueeze(0).to(device)
        )[0]
  _, predicted = torch.max(result, 1)
  print(test_dataset.idx_to_label[int(predicted)], end="\t")
  print(test_segmented[i])

POS	Quần mềm Lên dáng đẹp Đóng_gói sản_phẩm rất đẹp và chắc_chắn Shop phục_vụ rất tốt Thời_gian giao hàng rất nhanh .
NEG	Shop phục_vụ không nhiệt_tình .
NEG	Chất áo rất xấu .
POS	Mong lần sau shop sẽ uy_tín hơn !
POS	Váy đẹp , lên dáng chuẩn chỉ mỗi cái phần eo hơi dặmmm .
NEU	Áo đẹp , tương_đối giống hình , vải mát nhg dễ nhăn , kiểu áo này cũng khó là ủi , đường chỉ hơi mỏng may , kém chắc chắc .
POS	Bạn shop n.c dễ_thương .
POS	Nói chùn là rất ưng ạ ❤️ .
POS	mới dùng thấy ok . đánh_giá shop 5 * .
NEG	Bị mẹ lấy luôn rồi .
POS	Sản_phẩm rất đẹp rất đáng tiền💋💋💋💋 .
NEG	Túi thơm mùi như mùi chất hoá_học , để 1 lúc đau hết cả đầu .
POS	Nói chug 10 saooooo .
NEU	Chất vải tạm được .
NEG	Hàng không giống hình , cổ may hơi chật , trong hình không có túi nhưng nhận hàng lại có túi làm xấu áo .
POS	Gối nằm rất êm .
POS	áo với quần đều đẹp ưng_ý mình chủ shop rất nhiệt_tình dễ_thương .
NEG	Chất_lượng sản_phẩm rất kém , áo quá mỏng , không đáng tiền , mình đã ib cho shop yêu_cầu huỷ đơn hàng , s

In [ ]:
# Measure F1-score
y_pred = []
y_true = []

model.eval()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# test_dataset[0]['input_ids'].shape

for i in range(len(test_dataset)):
  result = model(input_ids=test_dataset[i]["input_ids"].unsqueeze(0).to(device),
        attention_mask=test_dataset[i]["attention_mask"].unsqueeze(0).to(device),
        token_type_ids=test_dataset[i]["token_type_ids"].unsqueeze(0).to(device)
        )[0]
  _, predicted = torch.max(result, 1)
  y_pred.append(int(predicted))
  y_true.append(int(test_dataset[i]["labels"]))

In [ ]:
import sklearn
sklearn.metrics.f1_score(y_true, y_pred, average='weighted')

0.7864696284107583

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 5349
  Batch size = 4


{'epoch': 1.0,
 'eval_loss': 0.48257994651794434,
 'eval_runtime': 159.9592,
 'eval_samples_per_second': 33.44,
 'eval_steps_per_second': 8.365}

In [ ]:
model_dir = '/content/drive/MyDrive/thesis/models'
trainer.save_model(model_dir + 'ecomm-review-phobert-base-latest')

Saving model checkpoint to /content/drive/My Drive/data/models/ecomm-review-phobert-base
Configuration saved in /content/drive/My Drive/data/models/ecomm-review-phobert-base/config.json
Model weights saved in /content/drive/My Drive/data/models/ecomm-review-phobert-base/pytorch_model.bin


In [ ]:
suc_khoe_comments_df = pd.read_csv("/content/suc-khoe-comments-100-sampled.csv")

suc_khoe_comments_segmented = [rdrsegmenter.tokenize(text) for text in suc_khoe_comments_df['0'].values]
def flatten(t):
    return [item for sublist in t for item in sublist]
suc_khoe_comments_segmented = [" ".join(flatten(t)) for t in suc_khoe_comments_segmented]

suc_khoe_comments_encodings = tokenizer(suc_khoe_comments_segmented, truncation=True, padding="max_length", max_length=256)

In [ ]:
print(suc_khoe_comments_segmented[0])
print(suc_khoe_comments_encodings["input_ids"][0])
print(suc_khoe_comments_encodings["attention_mask"][0])
print(suc_khoe_comments_encodings["token_type_ids"][0])

Mong phép màu đến với con . Thương con nhiều quá , cố gắn chiến_đấu vs nó con nhé ! mong tất_cả những tốt_đẹp đến bên con .
[0, 7675, 996, 412, 30, 15, 73, 5, 3999, 73, 36, 204, 4, 2105, 1079, 1185, 7136, 231, 73, 2083, 381, 1790, 392, 21, 2296, 30, 145, 73, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
model.eval()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# test_dataset[0]['input_ids'].shape

comment_y_true = []
comment_y_pred = []
label_to_idx = {
    "NEG": 0,
    "POS": 1,
    "NEU": 2
}

for i in range(len(suc_khoe_comments_encodings["input_ids"])):
  input_ids = torch.IntTensor(suc_khoe_comments_encodings["input_ids"][i]).unsqueeze(0).to(device)
  attention_mask = torch.IntTensor(suc_khoe_comments_encodings["attention_mask"][i]).unsqueeze(0).to(device)
  token_type_ids = torch.IntTensor(suc_khoe_comments_encodings["token_type_ids"][i]).unsqueeze(0).to(device)

  result = model(input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids=token_type_ids
        )[0]
  _, predicted = torch.max(result, 1)
  comment_y_pred.append(int(predicted))
  comment_y_true.append(label_to_idx[suc_khoe_comments_df.iloc[i][1]])
  # print(test_dataset.idx_to_label[int(predicted)], end="\t")
  # print(suc_khoe_comments_segmented[i])
  # print(result, end="\n\n")

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(comment_y_true, comment_y_pred))

              precision    recall  f1-score   support

           0       0.38      0.82      0.51        11
           1       0.59      0.92      0.72        26
           2       0.91      0.51      0.65        63

    accuracy                           0.65       100
   macro avg       0.62      0.75      0.63       100
weighted avg       0.77      0.65      0.65       100

